In [ ]:
# documentation for NASDAQ ITCH data: 
# http://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHspecification.pdf

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# for day in date_range : 
for ticker in nasdaq_tickers :
    itch_data = pd.read_csv(FOLDER + DATE + '_' + ticker + '.csv') # folder and date 
    

In [184]:
# check to make sure the ask and bid depth match the number of orders we have recorded
def verify_book(book, index, row) :
    side = book.groupby('side')
    bid_ask_depth = side.apply(lambda df: np.sum(df.shares))
    if (bid_ask_depth[0] != row['bid depth']) or (bid_ask_depth[1] != row['ask depth']) :
        raise Exception(f'Parsing error. Recorded orders don\'t match the data. Error occured on line {index}')
# this function worked well for debugging at the beginning, but later it becomes hard 
# to use because prices that were once ask prices can later become bid prices, making 
# ask and bid depth harder to compute

In [6]:
def init_book_shape(smallest_bin, exp_growth_factor, N) :
    bins = [-smallest_bin * (exp_growth_factor ** n) for n in reversed(range(N))] + [0] + [smallest_bin * (exp_growth_factor ** n) for n in range(N-1)]
    bins.append('shares')
    bins.append('price')
    book_shape = pd.DataFrame(columns=bins)
    bins.pop()
    bins.pop()
    bins = np.asarray(bins)
    return [book_shape, bins]

In [28]:
def get_bin(order_price, price) :
    cap = smallest_bin * (exp_growth_factor ** N-1)
    percent_dif = (order_price - price) / price
    if percent_dif <= cap :
        try :
            return np.where(percent_dif > bins)[0][-1]
        except IndexError :
            return None

In [8]:
itch_data = pd.read_csv('010416_CPN.csv')

/home/ndriggs/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
book = pd.DataFrame([[' B', 0, 0],[' S', 0, 0]], columns=['side','price','shares']) # initialize with zero asks and bids 
smallest_bin = 0.001
exp_growth_factor = 2 
N = 10
book_shape, bins = init_book_shape(smallest_bin, exp_growth_factor, N)

first_u = True # Order replace messages ('U' for update) come in pairs, the first being a 
               # deletion of previous order and the second adding a new order to the book

for index, row in itch_data.iterrows() :
    order_type = row['type'] 
    if order_type == 'P' :
        continue # Trade messages (non-cross) do not affect the book (see the docs)
    elif order_type == 'A' or order_type == 'F' or (order_type == 'U' and not first_u) : # adding an order to the book
        try :
            book.iloc[np.where(book['price'] == row['price'])[0][0], 2] += row['shares']
        except IndexError :
            book.loc[len(book)] = [row['side']] + [row['price']] + [row['shares']]
        if order_type == 'U' :
            first_u = True
    elif order_type == 'D' or order_type == 'X' or (order_type == 'U' and first_u): # deleting an order
        book.iloc[np.where(book['price'] == row['price'])[0][0], 2] -= row['shares']
        if order_type == 'U' :
            first_u = False
    elif order_type == 'E' or order_type == 'C': # order was executed
        try :
            last_price = book_shape.iloc[-1, -1] # price the last transaction was executed at
        except IndexError :
            last_price = row['price'] # at start of trading day there is no last transaction, use price of this transaction
        book_shape_temp = [0] * 2*N
        for i in range(len(book)) :
        # compute how far it is from the price
            byn = get_bin(book.loc[i, 'price'], last_price)
            if byn is not None :
                # add that many shares to its bin
                book_shape_temp[byn] += book.loc[i, 'shares']
        if row['side'] == ' S' :
            book_shape_temp.append(row['shares'])
        else :
            book_shape_temp.append(-1 * row['shares'])
        book_shape_temp.append(row['price']) # price that this trade executed at 
        new_book_shape_row = pd.DataFrame([book_shape_temp], columns = book_shape.columns)
        book_shape = pd.concat([book_shape, new_book_shape_row], ignore_index=True)
        book_shape.reset_index()
        # update book
        if order_type == 'E' :
            book.iloc[np.where(book['price'] == row['price'])[0][0], 2] -= row['shares']
        else : # 'C' indicates that the trade was executed at a price not listed on the book, 
               # somewhere in the spread. We will approximate it as the lowest ask or the highest bid
            price_row = np.where(book['price'] == row['price'])[0][0] 
            while book.iloc[price_row, 2] == 0 : 
                if row['side'] == ' S' : # market buy, we want to find lowest ask 
                    price_row += 1
                else :
                    price_row -= 1
            book.iloc[price_row, 2] -= row['shares']
    else :
        raise Exception(f'Incorrect order type. Order type was {order_type}')
    book.sort_values(by='price', inplace=True, ignore_index=True)
    if index == 10000 :
        break
    #verify_book(book, index, row) # check to make sure the ask and bid depth match the number of orders we have recorde

In [37]:
book_shape.iloc[30:50, 1:20]

,-0.256,-0.128,-0.064,-0.032,-0.016,-0.008,-0.004,-0.002,-0.001,0,0.001,0.002,0.004,0.008,0.016,0.032,0.064,0.128,0.256
30,2900,600,11900,13400,3776,1700,500,100,340,200,200,600,900,2800,12976,11910,500,3000,60
31,2900,600,11900,13400,3776,1700,500,200,468,200,200,700,900,2800,12976,11910,500,3000,60
32,2900,600,11900,13900,4076,1000,600,368,260,200,200,600,900,2700,12976,11910,500,3000,60
33,2900,600,11900,13900,4076,1000,600,368,200,200,200,600,900,2700,12976,11910,500,3000,60
34,2900,600,11900,13900,3376,900,1300,300,468,100,0,600,1000,2700,12976,11910,500,3000,60
35,2900,600,11900,13900,3376,1700,800,268,300,0,300,500,1200,9300,5976,12010,400,3000,60
36,2900,600,11900,13900,3276,1800,1000,200,668,0,100,206,1600,8300,6976,12010,400,3000,60
37,2900,600,17600,8200,3276,1800,1000,368,600,100,100,600,2200,14000,276,12010,400,3000,60
38,2900,600,17600,7700,3576,1200,1468,200,100,200,206,500,2100,13300,1276,12010,400,3000,60
39,2900,600,17600,7700,3576,1200,1400,300,268,200,200,500,2100,13300,1276,12010,400,3000,60
